In [1]:
'''The example demonstrates how to write custom layers for Keras.

We build a custom activation layer called 'Antirectifier',
which modifies the shape of the tensor that passes through it.

We need to specify two methods: `compute_output_shape` and `call`.
Note that the same result can also be achieved via a Lambda layer.

Because our custom layer is written with primitives from the Keras
backend (`K`), our code can run both on TensorFlow and Theano.
'''

"The example demonstrates how to write custom layers for Keras.\n\nWe build a custom activation layer called 'Antirectifier',\nwhich modifies the shape of the tensor that passes through it.\n\nWe need to specify two methods: `compute_output_shape` and `call`.\nNote that the same result can also be achieved via a Lambda layer.\n\nBecause our custom layer is written with primitives from the Keras\nbackend (`K`), our code can run both on TensorFlow and Theano.\n"

In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from keras import layers

Using TensorFlow backend.


In [3]:
class Antirectifier(layers.Layer):
    '''This is the combination of a sample-wise
    L2 normalization with the concatenation of the
    positive part of the input with the negative part
    of the input. The result is a tensor of samples that are
    twice as large as the input samples.
    It can be used in place of a ReLU.
    # Input shape
        2D tensor of shape (samples, n)
    # Output shape
        2D tensor of shape (samples, 2*n)
    # Theoretical justification
        When applying ReLU, assuming that the distribution
        of the previous output is approximately centered around 0.,
        you are discarding half of your input. This is inefficient.
        Antirectifier allows to return all-positive outputs like ReLU,
        without discarding any data.
        Tests on MNIST show that Antirectifier allows to train networks
        with twice less parameters yet with comparable
        classification accuracy as an equivalent ReLU-based network.
    '''

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        assert len(shape) == 2  # only valid for 2D tensors
        shape[-1] *= 2
        return tuple(shape)

    def call(self, inputs):
        inputs -= K.mean(inputs, axis=1, keepdims=True)
        inputs = K.l2_normalize(inputs, axis=1)
        pos = K.relu(inputs)
        neg = K.relu(-inputs)
        return K.concatenate([pos, neg], axis=1)

In [4]:
# Variables initialization
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [5]:
# Load data
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Good source for datasets
# https://s3.amazonaws.com/img-datasets/mnist.npz

In [6]:
# Preprocess data
'''
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
'''

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 784)
60000 train samples
10000 test samples


In [7]:
# Model Building

# build the model
model = Sequential()
model.add(layers.Dense(256, input_shape=(784,)))
model.add(Antirectifier())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256))
model.add(Antirectifier())
model.add(layers.Dropout(0.1))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# train the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

# next, compare with an equivalent network
# with2x bigger Dense layers and ReLU
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 6s 101us/step - loss: 0.6049 - acc: 0.9161 - val_loss: 0.1658 - val_acc: 0.9577
Epoch 2/12
60000/60000 [==============================] - 3s 46us/step - loss: 0.1254 - acc: 0.9661 - val_loss: 0.0930 - val_acc: 0.9730
Epoch 3/12
60000/60000 [==============================] - 3s 47us/step - loss: 0.0820 - acc: 0.9765 - val_loss: 0.0803 - val_acc: 0.9749
Epoch 4/12
60000/60000 [==============================] - 3s 47us/step - loss: 0.0629 - acc: 0.9813 - val_loss: 0.0762 - val_acc: 0.9765
Epoch 5/12
60000/60000 [==============================] - 3s 47us/step - loss: 0.0505 - acc: 0.9847 - val_loss: 0.0703 - val_acc: 0.9789
Epoch 6/12
60000/60000 [==============================] - 3s 48us/step - loss: 0.0426 - acc: 0.9873 - val_loss: 0.0738 - val_acc: 0.9783
Epoch 7/12
60000/60000 [==============================] - 3s 46us/step - loss: 0.0351 - acc: 0.9896 - val_loss: 0.0746 - val_ac

In [9]:
# Model Summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
antirectifier_1 (Antirectifi (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
antirectifier_2 (Antirectifi (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
__________